In [1]:
from option import args
import torch
import utility
import data
import utils
import model
import loss
from trainer import Trainer
import os
from torch.utils.tensorboard import SummaryWriter
import numpy as np

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


number of common Subjects  171


In [6]:
args.no_vols = 2
args.test_vols = 2
args.tv_en = True
np.random.seed(args.seed)
ids = utils.get_ids()
# ids.sort()
total_vols = args.no_vols+args.test_vols
ids.sort()
ids = ids[:total_vols]
ids = np.random.choice(ids,total_vols,replace = False)

In [7]:
ids

array(['104416', '102816', '100610', '102311'], dtype='<U6')

In [8]:
loader = data.Data(args,ids= ids)

Loading Done


In [9]:
for batch, (lr,hr,scale,tv) in enumerate(loader.training_data):
    lr_tensor = lr.squeeze().to('cuda').float()  # ranges from [0, 1]
    hr_tensor = hr.squeeze().to('cuda').float()  # ranges from [0, 1]
#     tv_tensor = tv.squeeze().to('cuda').float()  # ranges from [0, 1]
    # print(lr_tensor.shape,hr_tensor.shape,scale)
    scale = np.asarray(scale[0,:])
    break

In [10]:
tv[0]

tensor([[[[[0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           ...,
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.]],

          [[0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           ...,
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.]],

          [[0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           ...,
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.]],

          ...,

          [[0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           ...,
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.]],

          [[0., 0., 0., 

In [14]:
tv[0].size()

torch.Size([1, 133, 177, 113, 6])

In [17]:
tv[1]

('100610',)

In [18]:
torch.histogram(tv[0])

torch.return_types.histogram(
hist=tensor([1.0000e+00, 0.0000e+00, 0.0000e+00, 2.6400e+02, 1.2294e+07, 3.1029e+06,
        3.9089e+05, 1.6731e+05, 5.5430e+03, 1.6000e+01, 1.3000e+01, 9.0000e+00,
        6.0000e+00, 0.0000e+00, 3.0000e+00, 0.0000e+00, 3.0000e+00, 3.0000e+00,
        3.0000e+00, 4.0000e+00, 2.0000e+00, 5.0000e+00, 4.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 2.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 2.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
     

In [10]:
tv[1]

('102311',)

In [17]:
tv[0].shape

torch.Size([1, 133, 177, 113, 6])

In [18]:
batch, (lr.shape,hr.shape,scale,tv.shape)

AttributeError: 'list' object has no attribute 'shape'

In [11]:
batch, (lr_tensor.shape,hr_tensor.shape,scale,tv_tensor.shape)

(0,
 (torch.Size([16, 32, 32, 4, 7]),
  torch.Size([16, 42, 42, 5, 5]),
  tensor([[1.2000, 1.1000, 1.1000]], dtype=torch.float64),
  torch.Size([16, 42, 42, 5, 6])))

In [22]:
np.unique(tv_tensor.cpu())

array([0.05404305, 0.05469534, 0.055117  , ..., 0.36352447, 0.36451533,
       0.39078027], dtype=float32)

In [1]:
from dipy.viz import window, actor